# Shared code block

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = (256, 256)

def load_images_from_folder(folder, size=IMAGE_SIZE):
    imgs = []
    files = sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
    ])

    for fname in files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Skipping unreadable: {fname}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        img = img.astype("float32") / 255.0
        imgs.append(img)

    return np.array(imgs)


def build_autoencoder(input_shape=(256, 256, 3)):
    inp = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    x = MaxPooling2D((2,2), padding="same")(x)

    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2), padding="same")(x)

    # Bottleneck
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)

    # Decoder
    x = UpSampling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)

    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, (3,3), activation="relu", padding="same")(x)

    out = Conv2D(3, (3,3), activation="sigmoid", padding="same")(x)

    model = Model(inp, out)
    model.compile(optimizer=Adam(1e-3), loss="mae")

    return model


In [2]:
# blur model -> working, needs small tuning
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)
print(input_images.shape)

print("Loading BLUR style images...")
blur_images = load_images_from_folder(STYLE_FOLDER)
print(blur_images.shape)

if len(input_images) != len(blur_images):
    raise ValueError("Mismatch in BLUR training set!")

model = build_autoencoder()
model.fit(
    input_images,
    blur_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_blur.keras")
print("Saved BLUR model.")


Loading input images...
(5, 256, 256, 3)
Loading BLUR style images...
(5, 256, 256, 3)
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2699 - val_loss: 0.3452
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 967ms/step - loss: 0.2505 - val_loss: 0.2909
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 998ms/step - loss: 0.2276 - val_loss: 0.2197
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 965ms/step - loss: 0.2101 - val_loss: 0.1800
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 973ms/step - loss: 0.2217 - val_loss: 0.1660
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 964ms/step - loss: 0.2007 - val_loss: 0.1893
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step - loss: 0.1851 - val_loss: 0.2044
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1768 - val_loss: 0.1811
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 963ms/step - loss: 0.1635 - val_loss: 0.1899
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 949ms/step - loss: 0.1557 - val_loss: 0.1318
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 962ms/step - loss: 0.1432 -

In [4]:
# night_vis

import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/night_vis"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading Night Vision style images...")
night_vis_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(night_vis_images):
    raise ValueError("Mismatch in Night Vision training set!")

model = build_autoencoder()
model.fit(
    input_images,
    night_vis_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_night_vis.keras")
print("Saved  Night Vision model.")


Loading input images...
Loading Night Vision style images...
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.3549 - val_loss: 0.2904
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step - loss: 0.3017 - val_loss: 0.2405
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step - loss: 0.2485 - val_loss: 0.2295
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.2178 - val_loss: 0.2400
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.2098 - val_loss: 0.2335
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1990 - val_loss: 0.2062
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step - loss: 0.1785 - val_loss: 0.1931
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 962ms/step - loss: 0.1657 - val_loss: 0.1990
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 936ms/step - loss: 0.1721 - val_loss: 0.2031
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step - loss: 0.1778 - val_loss: 0.2022
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 980ms/step - loss: 0.1762 - val_loss: 0.1982
Epoch 12/200
1

In [5]:

# outline
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/outline"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading OUTLINE style images...")
outline_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(outline_images):
    raise ValueError("Mismatch in OUTLINE training set!")

model = build_autoencoder()
model.fit(
    input_images,
    outline_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_outline.keras")
print("Saved OUTLINE model.")


Loading input images...
Loading OUTLINE style images...
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2768 - val_loss: 0.3450
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.2393 - val_loss: 0.2287
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step - loss: 0.1971 - val_loss: 0.1252
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step - loss: 0.2004 - val_loss: 0.1303
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 953ms/step - loss: 0.2004 - val_loss: 0.1641
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1942 - val_loss: 0.1953
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 986ms/step - loss: 0.1866 - val_loss: 0.2166
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step - loss: 0.1884 - val_loss: 0.1911
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 996ms/step - loss: 0.1820 - val_loss: 0.1747
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 948ms/step - loss: 0.1793 - val_loss: 0.1714
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step - loss: 0.1759 - val_loss: 0.1819
Epoch 12/200
1/1

In [6]:
# poster style
import os

BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_FOLDER = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/poster"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models"

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("Loading input images...")
input_images = load_images_from_folder(BASE_INPUT)

print("Loading POSTER style images...")
poster_images = load_images_from_folder(STYLE_FOLDER)

if len(input_images) != len(poster_images):
    raise ValueError("Mismatch in POSTER training set!")

model = build_autoencoder()
model.fit(
    input_images,
    poster_images,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

model.save(f"{MODEL_SAVE_DIR}/autoencoder_poster.keras")
print("Saved POSTER model.")


Loading input images...
Loading POSTER style images...
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2730 - val_loss: 0.3561
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 916ms/step - loss: 0.2666 - val_loss: 0.3282
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 957ms/step - loss: 0.2556 - val_loss: 0.2885
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 958ms/step - loss: 0.2380 - val_loss: 0.2378
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 980ms/step - loss: 0.2234 - val_loss: 0.2421
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step - loss: 0.2215 - val_loss: 0.2611
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 975ms/step - loss: 0.2071 - val_loss: 0.2162
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 978ms/step - loss: 0.1847 - val_loss: 0.1974
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 940ms/step - loss: 0.1738 - val_loss: 0.2238
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step - loss: 0.1641 - val_loss: 0.2250
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.1553 - val_loss: 0.2039
Epoch 12/200
1